In [ ]:
# pega o df_final do arquivo main pra trabalhar com sql
import pandas as pd
from main import df_final
import sqlite3
# cria uma conexão com o banco de dados SQLite (ou cria o banco se não existir)
conn = sqlite3.connect('banco_de_dados.db')
# exporta o DataFrame para uma tabela SQL chamada 'tabela_final'
df_final.to_sql('tabela_final', conn, if_exists='replace', index=False)
# fecha a conexão com o banco de dados
conn.close()

PySparkRuntimeError: [JAVA_GATEWAY_EXITED] Java gateway process exited before sending its port number.